Sample code for Greedy Best First Search, you guys can add code blocks to test other codes.

In [ ]:
class Ronny:
  def __init__(self, x, y, trashSize, trashWeight):
    self.x = x
    self.y = y
    self.trashSize = 0
    self.trashWeight = 0

  def addTrashSize(self, size):
    self.trashSize += size

  def addTrashWeight(self, weight):
    self.trashWeight += weight

class Cell:
  def __init__(self, x, y, trashSize, trashWeight):
    self.x = x
    self.y = y
    self.trashSize = trashSize
    self.trashWeight = trashWeight

class Node:
  def __init__(self, ronny: Ronny, cells: list[Cell], parent=None, action=None):
    self.ronny = ronny
    self.cells = cells
    self.parent = parent
    self.action = action

def getNodeState(node):
  if node is not None:
    return [node.ronny.x, node.ronny.y, node.ronny.trashSize, node.ronny.trashWeight, node.action]

def transitionModel(node, action):
  def isWithinMaze(child):
    if child.ronny.y % 2 == 0:
      return child.ronny.x in range(1,12,2) and child.ronny.y in range(0,9)
    else:
      return child.ronny.x in range(0,11,2) and child.ronny.y in range(0,9)

  def isOverflow(child):
    return child.ronny.trashSize > 5 or child.ronny.trashWeight > 40

  def returnTrash(x,y):
    for cell in child.cells:
      if cell.x == x and cell.y == y:
        return (cell.trashSize, cell.trashWeight)
      else:
        return -1

  child = Node(Ronny(node.ronny.x, node.ronny.y, node.ronny.trashSize, node.ronny.trashWeight), node.cells, node, action)
  if action == "NE":
    child.ronny.x -= 1
    child.ronny.y += 1
  elif action == "SE":
    child.ronny.x += 1
    child.ronny.y += 1
  elif action == "S":
    child.ronny.x += 2
  elif action == "SW":
    child.ronny.x += 1
    child.ronny.y -= 1
  elif action == "NW":
    child.ronny.x -= 1
    child.ronny.y -= 1
  elif action == "N":
    child.ronny.x -= 2

  if returnTrash(child.ronny.x, child.ronny.y) != -1:
    child.ronny.addTrashSize(returnTrash(child.ronny.x, child.ronny.y)[0])
    child.ronny.addTrashWeight(returnTrash(child.ronny.x, child.ronny.y)[1])

  if isWithinMaze(child) and not(isOverflow(child)):
    return child

def calculateHeuristic(ronny, cells):
  pass

def gbfs(node):
  actions = ["NE", "SE", "S", "SW", "NW", "N"]
  frontier = []
  explored = []
  frontier.append(node)

  children = []
  for action in actions:
    transition = transitionModel(frontier[0], action)
    if transition is not None:
      children.append(transition)
  explored.append(frontier[0])
  del frontier[0]

  for child in children:
    # still need to add in check condition if it is explored state
    frontier.append(child)

  print("Explored: ", [getNodeState(e) for e in explored])
  print("Frontier: ", [getNodeState(f) for f in frontier])
  print("Children: ", [getNodeState(c) for c in children])

if __name__ == "__main__":
  cells = [Cell(11,0,10,1), Cell(6,1,30,3), Cell(5,2,5,1), Cell(11,2,0,0),
           Cell(2,3,5,1), Cell(8,3,5,3), Cell(5,4,10,2), Cell(9,4,20,1),
           Cell(0,5,0,0), Cell(3,6,10,2), Cell(9,6,5,2), Cell(0,7,30,1),
           Cell(6,7,20,2), Cell(3,8,10,3), Cell(11,8,0,0), Cell(0,1,50,0)] # added extra cell at the back for testing

  initial_ronny = Ronny(1,0,0,0)

  gbfs(Node(initial_ronny, cells))

Explored:  [[1, 0, 0, 0, None]]
Frontier:  [[0, 1, 0, 0, 'NE'], [2, 1, 0, 0, 'SE'], [3, 0, 0, 0, 'S']]
Children:  [[0, 1, 0, 0, 'NE'], [2, 1, 0, 0, 'SE'], [3, 0, 0, 0, 'S']]


Redesigned GBFS (Greedy Best First Search)...

Chooses the closest eligible cell (trash cell/disposal rooms)

Also incorporates bias for Ronny to fill up his bin before disposing (not refined yet)

If more than one choice have the same heuristic function, it chooses the first in the list (not optimal)

In [ ]:
class Node:
  def __init__(self, ronny_x, ronny_y, ronny_ts, ronny_tw, cells, parent=None, action=None, evaluation=None):
    self.ronny_x = ronny_x
    self.ronny_y = ronny_y
    self.ronny_ts = ronny_ts
    self.ronny_tw = ronny_tw
    self.cells = cells # list of tuples
    self.parent = parent
    self.action = action
    self.evaluation = evaluation

  def addTrash(self, size, weight):
      self.ronny_ts += size
      self.ronny_tw += weight

  def updateCells(self, trash):
    self.cells.remove(trash)

def getNodeState(node):
    return [node.ronny_x, node.ronny_y, node.ronny_ts, node.ronny_tw, node.action, f"Cells left: {len(node.cells)}", node.evaluation]

def transitionModel(node, action):
  def isWithinMaze(node):
    if node.ronny_y % 2 == 0:
      return node.ronny_x in range(1,12,2) and node.ronny_y in range(0,9)
    else:
      return node.ronny_x in range(0,11,2) and node.ronny_y in range(0,9)

  def isOverflow(node):
    return node.ronny_ts > 5 or node.ronny_tw > 40

  child = Node(node.ronny_x, node.ronny_y, node.ronny_ts, node.ronny_tw, node.cells.copy(), node, action)
  if action == "NE":
    child.ronny_x -= 1
    child.ronny_y += 1
  elif action == "SE":
    child.ronny_x += 1
    child.ronny_y += 1
  elif action == "S":
    child.ronny_x += 2
  elif action == "SW":
    child.ronny_x += 1
    child.ronny_y -= 1
  elif action == "NW":
    child.ronny_x -= 1
    child.ronny_y -= 1
  elif action == "N":
    child.ronny_x -= 2

  cell = next((cell for cell in child.cells if cell[0] == child.ronny_x and cell[1] == child.ronny_y), None)
  # check if there is a cell where Ronny is
  if cell is not None:
    # identifies that it is a trash cell
    if cell[2] != 0 and cell[3] != 0:
      child.addTrash(cell[2], cell[3])
      child.updateCells(cell)
    #identifies that it is a disposal cell
    else:
      child.ronny_ts, child.ronny_tw = 0, 0

  if isWithinMaze(child) and not(isOverflow(child)):
    return child

def calculateEvaluation(node):
  eligible_cells = []
  disposal_bias = min(5 / (node.ronny_ts + 0.0001), 40 / (node.ronny_tw + 0.0001))
  evaluation_list = []

  # populate eligible cells
  for cell in node.cells:
    if cell[2] + node.ronny_ts <= 5 and cell[3] + node.ronny_tw <= 40:
      eligible_cells.append(cell)

  for cell in eligible_cells:
    trash_bias = 1.0001 - max((node.ronny_ts + cell[2]) / 5, (node.ronny_tw + cell[3]) / 40)
    if cell[2] == 0 and cell[3] == 0:
      evaluation_list.append(((cell[0] - node.ronny_x) ** 2 + (cell[1] - node.ronny_y) ** 2) ** 1/2 * disposal_bias)
    else:
      evaluation_list.append(((cell[0] - node.ronny_x) ** 2 + (cell[1] - node.ronny_y) ** 2) ** 1/2 * trash_bias)
  evaluation = min(evaluation_list)
  return round(evaluation, 4)

def gbfs(node):
  actions = ["NE", "SE", "S", "SW", "NW", "N"]
  frontier = []
  explored = []
  frontier.append(node)

  # loop till no trash cells and ronny has no trash
  while len(frontier[0].cells) >= 3:
    if len(frontier[0].cells) == 3 and (frontier[0].ronny_ts,frontier[0].ronny_tw) == (0,0):
      break
    children = []
    for action in actions:
      transition = transitionModel(frontier[0], action)
      if transition is not None:
        children.append(transition)
    explored.append(frontier[0])
    del frontier[0]

    for child in children:
      # check condition if it is explored state (not sure if correct, needs testing)
      if not((child.ronny_x, child.ronny_y, child.cells) in [(e.ronny_x, e.ronny_y, e.cells) for e in explored]):
        child.evaluation = calculateEvaluation(child)
        frontier.append(child)
    frontier.sort(key = lambda node:(len(node.cells), node.evaluation))

    print("Explored: ", [getNodeState(e) for e in explored])
    print("Frontier: ", [getNodeState(f) for f in frontier])
    print("Children: ", [getNodeState(c) for c in children])

    # choosing the path where evaluation and cells left are lowest
    while frontier[-1].evaluation > frontier[0].evaluation or len(frontier[-1].cells) > len(frontier[0].cells):
      frontier.pop(-1)
    print("Final Frontier: ", [getNodeState(f) for f in frontier])
    print()

  solution = [frontier[0].action]
  total_step = 0
  while frontier[0].parent:
    if frontier[0].parent.action:
      solution.insert(0, frontier[0].parent.action)
    frontier[0] = frontier[0].parent
    total_step += 1
  return solution, total_step

if __name__ == "__main__":
  cells = [(11,0,1,10), (6,1,3,30), (5,2,1,5), (11,2,0,0),
           (2,3,1,5), (8,3,3,5), (5,4,2,10), (9,4,1,20),
           (0,5,0,0), (3,6,2,10), (9,6,2,5), (0,7,1,30),
           (6,7,2,20), (3,8,3,10), (11,8,0,0)]

  initial_node = Node(1,0,0,0,cells.copy())
  [solution, total_step] = gbfs(initial_node)
  print("Solution: ", solution)
  print("Total_steps: ", total_step)

Explored:  [[1, 0, 0, 0, None, 'Cells left: 15', None]]
Frontier:  [[3, 0, 0, 0, 'S', 'Cells left: 15', 1.2505], [2, 1, 0, 0, 'SE', 'Cells left: 15', 1.6002], [0, 1, 0, 0, 'NE', 'Cells left: 15', 3.2004]]
Children:  [[0, 1, 0, 0, 'NE', 'Cells left: 15', 3.2004], [2, 1, 0, 0, 'SE', 'Cells left: 15', 1.6002], [3, 0, 0, 0, 'S', 'Cells left: 15', 1.2505]]
Final Frontier:  [[3, 0, 0, 0, 'S', 'Cells left: 15', 1.2505]]

Explored:  [[1, 0, 0, 0, None, 'Cells left: 15', None], [3, 0, 0, 0, 'S', 'Cells left: 15', 1.2505]]
Frontier:  [[5, 0, 0, 0, 'S', 'Cells left: 15', 0.2501], [4, 1, 0, 0, 'SE', 'Cells left: 15', 0.5002], [2, 1, 0, 0, 'NE', 'Cells left: 15', 1.6002]]
Children:  [[2, 1, 0, 0, 'NE', 'Cells left: 15', 1.6002], [4, 1, 0, 0, 'SE', 'Cells left: 15', 0.5002], [5, 0, 0, 0, 'S', 'Cells left: 15', 0.2501], [1, 0, 0, 0, 'N', 'Cells left: 15', None]]
Final Frontier:  [[5, 0, 0, 0, 'S', 'Cells left: 15', 0.2501]]

Explored:  [[1, 0, 0, 0, None, 'Cells left: 15', None], [3, 0, 0, 0, 'S', 'C

In [ ]:
class Ronny:
    def __init__(self, rubbishSize=0, rubbishWeight=0):
        self.rubbishSize = rubbishSize
        self.rubbishWeight = rubbishWeight

    def addRubbish(self, rubbishSize, rubbishWeight):
        self.rubbishSize += rubbishSize
        self.rubbishWeight += rubbishWeight

    def dispose(self):
        self.rubbishSize = 0
        self.rubbishWeight = 0

    def hasRubbish(self):
        if self.rubbishSize != 0 or self.rubbishWeight != 0:
            return True
        else:
            return False

class SpecialCell:
    def __init__(self, x, y, rubbishSize, rubbishWeight):
        self.x = x
        self.y = y
        self.rubbishSize = rubbishSize
        self.rubbishWeight = rubbishWeight

class Node:
    def __init__(self, x, y, ronny: Ronny, special_cells: [SpecialCell], goal:SpecialCell = None):
        self.x = x      # Ronny's x position
        self.y = y      # Ronnt's y position
        self.ronny = ronny
        self.special_cells = special_cells
        self.goal = goal

    def totalRubbish(self):
        disposal_rooms = 0
        for cell in self.special_cells:
            if cell.rubbishSize == 0 and cell.rubbishWeight == 0:
                disposal_rooms += 1
        total_rubbish_cells = len(self.special_cells) - disposal_rooms
        return total_rubbish_cells

    def updateGoal(self, goal):
        self.goal = goal

    def state(self):
        total_rubbish_cells = self.totalRubbish()
        return [f"Ronny is at {self.x},{self.y}, carrying {self.ronny.rubbishSize} m3 and {self.ronny.rubbishWeight} kg of rubbish.", \
                      f"Number of Rubbish Cells Remaining: {total_rubbish_cells}"]

    def goalReached(self):
        if self.x == self.goal.x and self.y == self.goal.y:
            if self.goal.rubbishSize != 0 and self.goal.rubbishWeight != 0:
                self.ronny.addRubbish(self.goal.rubbishSize, self.goal.rubbishWeight)
                self.special_cells.remove(self.goal)
                self.goal = None
            else:
                self.ronny.dispose()
                self.goal = None

def selectGoal(node):
    weightage = .5
    h = []  # List of h values
    for cell in node.special_cells: # Heuristic used for goal selection
        dy = abs(node.y - cell.y)
        dx = abs(node.x - cell.x)
        distance = dx + max(0, (dy-dx)/2)   # Distance formula from https://www.redblobgames.com/grids/hexagons/
        totalSize = node.ronny.rubbishSize + cell.rubbishSize
        totalWeight = node.ronny.rubbishWeight + cell.rubbishWeight
        if totalSize > 5 or totalWeight > 40:
            h.append(999999999) # "infinity"
        else:
            h.append(int(distance + max((5 * weightage /(totalSize + .0001)),(40 * weightage /(totalWeight + .0001)))))

    min_h = h[0]
    # print([(cell.x,cell.y,h[node.special_cells.index(cell)]) for cell in node.special_cells])

    for i in range (0,len(h)):
        min_h = min(min_h, h[i])

    node.updateGoal(node.special_cells[h.index(min_h)])
    print()
    print(f"Current Goal: {node.goal.x,node.goal.y}")
    print()

def transitionalModel(action):
    if action == "N":
        dx = 0
        dy = -2
    if action == "S":
        dx = 0
        dy = 2
    if action == "NE":
        dx = 1
        dy = -1
    if action == "SE":
        dx = 1
        dy = 1
    if action == "NW":
        dx = -1
        dy = -1
    if action == "SW":
        dx = -1
        dy = 1
    return [dx,dy]

def optimalMove(node, maze_width, maze_height):
    actions = ["N", "S", "NE", "SE", "NW", "SW"]
    h = [] # List of h values
    possibleActions = [] # List of all possible actions
    for action in actions:
        [dx,dy] = transitionalModel(action)
        if (node.x + dx < 0) or (node.x + dx > maze_width) or (node.y + dy < 0) or (node.y + dy > maze_height):
            # Checking if actions will lead to Ronny to leave the confines of the maze
            continue
        else:
            # Check if action will put Ronny in our goal cell
            if (node.x + dx == node.goal.x) and (node.y + dy == node.goal.y):
                node.x += dx
                node.y += dy
                node.goalReached()
                return action
            else:
                if ((node.x+dx, node.y+dy) in [(cell.x,cell.y) for cell in node.special_cells]):
                    continue
                else:
                    possibleActions.append(action)
                    hdx = abs(node.x+dx - node.goal.x)
                    hdy = abs(node.y+dy - node.goal.y)
                    distance = hdx + max(0, (hdy-hdx)/2)
                    h.append(distance)
    min_h = h[0]

    for i in range (0,len(h)):
        min_h = min(min_h, h[i])

    optimalAction = possibleActions[h.index(min_h)]
    [dx,dy] = transitionalModel(optimalAction)
    node.x += dx
    node.y += dy
    return optimalAction


def gbfs(node, size):
    [maze_width, maze_height] = size
    goal_changes = 0
    step = 0
    listOfActions = []
    print("Step 0:" + str(node.state()))
    while node.ronny.hasRubbish() or node.totalRubbish() != 0:
        step += 1
        if node.goal == None:
            goal_changes += 1
            selectGoal(node)
        listOfActions.append(optimalMove(node, maze_width, maze_height))
        print(f"Step {step}:" + str(node.state()))
    print()
    print(listOfActions)
    print("Total Actions: %d" % len(listOfActions))
    print("Total Goal Changes: %d" % goal_changes)

def main():
    maze_size = [8,11]  # w * h
    maze = Node(0,1,Ronny(),[SpecialCell(0,11,1,10), SpecialCell(1,6,3,30), SpecialCell(2,5,1,5),\
                             SpecialCell(2,11,0,0), SpecialCell(3,2,1,5), SpecialCell(3,8,3,5),\
                             SpecialCell(4,5,2,10), SpecialCell(4,9,1,20), SpecialCell(5,0,0,0),\
                             SpecialCell(6,3,2,10), SpecialCell(6,9,2,5), SpecialCell(7,0,1,30),\
                             SpecialCell(7,6,2,20), SpecialCell(8,3,3,10), SpecialCell(8,11,0,0)])
    gbfs(maze, maze_size)

if __name__ == '__main__':
    main()

Step 0:['Ronny is at 0,1, carrying 0 m3 and 0 kg of rubbish.', 'Number of Rubbish Cells Remaining: 12']

Current Goal: (1, 6)

Step 1:['Ronny is at 0,3, carrying 0 m3 and 0 kg of rubbish.', 'Number of Rubbish Cells Remaining: 12']
Step 2:['Ronny is at 0,5, carrying 0 m3 and 0 kg of rubbish.', 'Number of Rubbish Cells Remaining: 12']
Step 3:['Ronny is at 1,6, carrying 3 m3 and 30 kg of rubbish.', 'Number of Rubbish Cells Remaining: 11']

Current Goal: (2, 5)

Step 4:['Ronny is at 2,5, carrying 4 m3 and 35 kg of rubbish.', 'Number of Rubbish Cells Remaining: 10']

Current Goal: (3, 2)

Step 5:['Ronny is at 2,3, carrying 4 m3 and 35 kg of rubbish.', 'Number of Rubbish Cells Remaining: 10']
Step 6:['Ronny is at 3,2, carrying 5 m3 and 40 kg of rubbish.', 'Number of Rubbish Cells Remaining: 9']

Current Goal: (5, 0)

Step 7:['Ronny is at 4,1, carrying 5 m3 and 40 kg of rubbish.', 'Number of Rubbish Cells Remaining: 9']
Step 8:['Ronny is at 5,0, carrying 0 m3 and 0 kg of rubbish.', 'Number of